# 0. Setup

In [105]:
import keras as keras
import pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import keras.optimizers as optimizers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization
from keras import models
from keras import regularizers
import tensorflow as tf
import math
import os

# 1. Hyperparameters

In [71]:
input_height = 69
input_width = 69
batch_size = 64

# TODO: find best parameters using the display_data_augmentation_sample jupyter notebook
rescale = True
if rescale:
    rescale_size=1./255
else:
    rescale_size=1
augmentation=True

rotation_range=40
width_shift_range=0.2
height_shift_range=0.1
shear_range=0.2
zoom_range=0.2
horizontal_flip=True
fill_mode='nearest'

In [106]:
# best loss function for multi-class classification, measures the distance between two probability distributions
# the probability distribution of the output of the network and the true distribution of the labels
loss_function='categorical_crossentropy'

metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
    tf.keras.metrics.Accuracy(name="accuracy", dtype=None)
]
optimizer='rmsprop'
optimizer_learning_rate=1e-4
epochs=100
batch_size=32
#regularizer=regularizers.l1_l2(l1=0.001, l2=0.001) # simultaneous l1 and l2, add 0.001*weight_coefficient_value + 0.001 * 1/2*weight^2

if optimizer == 'rmsprop':
    optimizer=optimizers.RMSprop(learning_rate=optimizer_learning_rate)

In [73]:

paths = {
    'TRAIN_PATH' : os.path.join('data','workspace', 'images', 'train'),
    'TEST_PATH' : os.path.join('data','workspace', 'images','test'),
    'EVAL_PATH' : os.path.join('data','workspace', 'images','eval'),
    'IMAGES_PATH': os.path.join('data','workspace','images','all'),
    'ANNOTATION_PATH': os.path.join('data','workspace','annotations','final_annotations.csv'),
    'LOG_DIR' : os.path.join('data','model', 'log_dir')
 }

annotations = pd.read_csv(paths['ANNOTATION_PATH'])

In [74]:
#get number of images for each class
number_per_class = annotations.groupby('Class').count()
imgs_per_class = number_per_class.to_dict()['Path']
print(imgs_per_class)

{0: 3461, 1: 6997, 2: 6292, 3: 349, 4: 1534, 6: 589, 7: 1121, 8: 906, 9: 519}


In [91]:
#compute number of images in training set
imgs_per_label = dict()
for i in range(9):
  path = os.path.join(paths['TRAIN_PATH'],str(i))
  #compute number of images for each folder representing a label
  n_images = len([f for f in os.listdir(path)if os.path.isfile(os.path.join(path, f))])
  imgs_per_label[i] = n_images

print(imgs_per_label)

{0: 2488, 1: 5017, 2: 4551, 3: 257, 4: 1122, 5: 415, 6: 797, 7: 646, 8: 381}


In [95]:
#weight computation

#number of classes
NUM_CLASSES = 9

#get number of total images
tot_images = sum(list(imgs_per_label.values()))
print(tot_images)

#dictionary storing weights for each class
#weight[i] = number_total_samples / number_total_classes * num_samples_class_i
weights = dict([ (class_label , tot_images/(NUM_CLASSES * n_images)) for class_label, n_images in imgs_per_label.items()])

print(weights)

15674
{0: 0.6999821364773133, 1: 0.3471308661661462, 2: 0.3826753582851144, 3: 6.7764807609165585, 4: 1.5521885521885521, 5: 4.196519410977242, 6: 2.1851387146242853, 7: 2.695906432748538, 8: 4.571011956838729}


In [96]:
# training set image data generator
from keras.preprocessing.image import ImageDataGenerator
if augmentation:
    train_datagen = ImageDataGenerator(
          rescale=rescale_size,
          rotation_range=rotation_range,
          width_shift_range=width_shift_range,
          height_shift_range=height_shift_range,
          shear_range=shear_range,
          zoom_range=zoom_range,
          horizontal_flip=horizontal_flip,
          fill_mode=fill_mode)
else:
    train_datagen = ImageDataGenerator(rescale=1./255)
    # to perform normalization we should never use information coming from the test set, only training set

train_dir=paths['TRAIN_PATH']

# TODO: Consider if the output should be normalized
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

Found 15674 images belonging to 9 classes.


In [97]:
# validation set image data generator
val_datagen = ImageDataGenerator(rescale=rescale_size) # it should not be augmented

validation_dir=paths['EVAL_PATH']

validation_generator = val_datagen.flow_from_directory(validation_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

Found 3919 images belonging to 9 classes.


# 3. Model creation

In [79]:
# Function used to create the CNN structure used for regression
def create_cnn(width, height, depth, num_classes,filters=(16, 32, 64)):
    inputShape = (height, width, depth)
    chanDim = -1
    model = models.Sequential()
    for (i, f) in enumerate(filters):
        # first CONV layer set appropriately
        if i == 0:
            model.add(Conv2D(f, (3, 3), activation="relu", padding="same", input_shape=inputShape))
        else:
            model.add(Conv2D(f, (3, 3), activation="relu", padding="same"))
        # size of the patches typically 3x3 or 5x5
        # determine if we need to change padding or stride, with padding = same we are able to center convolutional windows around every input tile, in order to have always the same size of the input image
        # model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2, 2)))
    # FLATTEN => FC => RELU => BN => DROPOUT
    model.add(Flatten())
    #model.add(Dropout(0.5))
    model.add(Dense(64, activation="relu")) # consider if we need to add this dense layer before with more units, such as 64 in order to shrink in two different stages, depends on the outpout size of flatten
    #model.add(BatchNormalization(axis=chanDim))
    model.add(Dense(num_classes, activation="softmax"))
    return model

model = create_cnn(input_width, input_height, 3, 9, (16,32))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 69, 69, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 34, 34, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 34, 34, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 17, 17, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 9248)              0         
                                                                 
 dense (Dense)               (None, 64)                5

In [80]:
# compile the model
model.compile(optimizer=optimizer,
              loss=loss_function,
              metrics=metrics)

# 4. Model training

In [102]:
n_images_eval = 0
for i in range(9):
  path = os.path.join(paths['EVAL_PATH'],str(i))
  #compute number of images in each eval folder and sum it up
  n_images_eval = n_images_eval + len([f for f in os.listdir(path)if os.path.isfile(os.path.join(path, f))])

print(n_images_eval)

3919


In [ ]:
# steps_per_epoch: number of batches to be drawn from the generator after assuming epoch over
# epochs: number of epochs
# validation_steps: how many batches to draw from the validation generator for evaluation

# TODO: set to the number of images we have
number_training = tot_images # TODO: contare elementi training cartella
number_eval = n_images_eval

history = model.fit(
      train_generator,
      steps_per_epoch=int(math.ceil((1. * number_training) / batch_size)),
      epochs=epochs,
      class_weight=weights,
      validation_data=validation_generator,
      validation_steps=int(math.ceil((1. * number_eval) / batch_size)))

Epoch 1/100
490/490 [==============================] - 107s 218ms/step - loss: 1.7102 - fn: 15266.0000 - fp: 333.0000 - tn: 125059.0000 - tp: 408.0000 - precision: 0.5506 - recall: 0.0260 - val_loss: 1.5945 - val_fn: 3799.0000 - val_fp: 90.0000 - val_tn: 31262.0000 - val_tp: 120.0000 - val_precision: 0.5714 - val_recall: 0.0306
Epoch 2/100
490/490 [==============================] - 88s 180ms/step - loss: 1.6568 - fn: 15143.0000 - fp: 423.0000 - tn: 124969.0000 - tp: 531.0000 - precision: 0.5566 - recall: 0.0339 - val_loss: 1.5509 - val_fn: 3675.0000 - val_fp: 175.0000 - val_tn: 31177.0000 - val_tp: 244.0000 - val_precision: 0.5823 - val_recall: 0.0623
Epoch 3/100
490/490 [==============================] - 87s 177ms/step - loss: 1.6159 - fn: 14904.0000 - fp: 444.0000 - tn: 124948.0000 - tp: 770.0000 - precision: 0.6343 - recall: 0.0491 - val_loss: 1.4825 - val_fn: 3587.0000 - val_fp: 181.0000 - val_tn: 31171.0000 - val_tp: 332.0000 - val_precision: 0.6472 - val_recall: 0.0847
Epoch 4/10

True


# 5. Visualization

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)

In [ ]:
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation ACC')
plt.legend()
plt.figure()

In [ ]:
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
# smooth curves if they look noisy
# replace each point with an exponential moving average of the previous points
def smooth_curve(points, factor=0.8):
  smoothed_points = []
  for point in points:
    if smoothed_points:
      previous = smoothed_points[-1]
      smoothed_points.append(previous * factor + point * (1 - factor))
    else:
      smoothed_points.append(point)
  return smoothed_points

In [ ]:
plt.plot(epochs,
         smooth_curve(acc), 'r', label='Smoothed training acc')
plt.plot(epochs,
         smooth_curve(val_acc), 'b', label='Smoothed validation acc')
plt.title('Training and validation MAE')
plt.legend()
plt.figure()

In [ ]:
plt.plot(epochs,
         smooth_curve(loss), 'r', label='Smoothed training loss')
plt.plot(epochs,
         smooth_curve(val_loss), 'b', label='Smoothed validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()
# display average, the model may improve even if not reflected

# 6. Early stopping

In [ ]:
import keras
import os

callbacks_list = [
        # interrupts training when accuracy has stopped improving accuracy on the validation set for at least 3+1=4 epochs
        keras.callbacks.EarlyStopping(
            monitor='acc', # should be part of the metrics specific during compilation
            patience=10,
        ),
        # save the current weights after every epoch
        #keras.callbacks.ModelCheckpoint(
        #    filepath=os.path.join(paths['MODELS'],'CNN_baseline.h5'),
        #    monitor='val_loss', # do not overwrite until val_loss is improved
        #    save_best_only=True
        #),
        # monitor the model's validation loss and reduce the LR when the validation loss has stopped improving, effective strategy to escape local minima
        #keras.callbacks.ReduceLROnPlateau(
        #    monitor='val_loss',
        #    factor=0.2, # divides LR by 5 when triggered
        #    patience=3 # called when stopped improving for 3 epochs
        #),
        #keras.callbacks.TensorBoard(
        #    log_dir=paths['LOG_DIR'],
        #    write_graph=True,
        #    histogram_freq=1 # record activation histograms every 1 epoch
        #)
]

In [ ]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=int(math.ceil((1. * number_training) / batch_size)),
      epochs=epochs,
      validation_data=validation_generator,
      callbacks=callbacks_list,
      validation_steps=int(math.ceil((1. * number_eval) / batch_size)))

# 7. Model testing

In [ ]:
test_dir=paths["TEST_PATH"]
test_datagen = ImageDataGenerator(rescale=rescale_size) # it should not be augmented

test_generator = test_datagen.flow_from_directory(test_dir, target_size=(input_width, input_height), batch_size=batch_size, class_mode='categorical')

# if performances are much wors than validation ones, during hyperparameter optimization (when done) the process has overfitted the validdation set, if so go to a more clear protocol such as Kfold CV
test_loss, test_acc = model.evaluate_generator(test_generator, steps=2)
print('test acc:', test_acc)
print('test loss:', test_loss)

In [ ]:
# TODO: is it balanced? If not consider ROC AUC for example, FPR, TPR, and others

# 8. Model exportation

In [ ]:
model.save("models/CNN_baseline_class_weights.h5")

# 9. Plot model as graph of layers

In [ ]:
from keras.utils import plot_model

In [ ]:
plot_model(model, show_shapes=True, to_file='model.png')